In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
import scipy

In [2]:
from utils import *
from hash_utils import digest
from game_model import game
from rp_model import computeRP
from variables import *
from computed import computedColumns
from initial_guess import make_initial_guess


In [3]:
# stuff for display

from IPython.lib.pretty import pretty, pprint

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 120)

In [4]:
# Load previous data

data = pd.read_pickle('./data/rp-data.pickle')
data.describe()
data.head()

,Level,RP,MS lvl,ModelRP,Difference,Freq1,FreqL,Inv,Berry1,BerryL,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,BerryD,IngD,Dupes,Amnt,Ing2P,Help skill bonus,RP Multiplier
count,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.000000,5465.0,5465.0,5465.0,5465.000000,5465.000000
mean,9.504483,558.335041,1.399634,558.326214,-0.008827,3999.432754,3863.991508,12.022507,28.004209,36.541812,108.899360,0.998525,0.989268,0.994979,1.001025,1.005672,0.206873,30.613036,53.119122,145.736139,1.0,0.0,0.0,0.989369,1.025429
std,6.251181,326.788079,0.898714,326.800586,0.795908,942.987550,1003.505055,3.315086,2.612509,6.863081,15.761472,0.291601,0.059411,0.045319,0.118243,0.118452,0.043644,32.873601,25.247767,54.726980,0.0,0.0,0.0,0.034974,0.066618
min,1.000000,245.000000,1.000000,245.050000,-9.450000,2200.000000,1489.118400,7.000000,24.000000,24.000000,90.000000,0.520000,0.890000,0.920000,0.800000,0.800000,0.130000,5.980000,25.000000,90.000000,1.0,0.0,0.0,0.790000,1.000000
25%,5.000000,349.000000,1.000000,348.870000,-0.310000,3400.000000,3079.788000,10.000000,26.000000,32.000000,98.000000,0.800000,1.000000,1.000000,1.000000,1.000000,0.180000,11.980000,35.000000,101.000000,1.0,0.0,0.0,1.000000,1.000000
50%,8.000000,436.000000,1.000000,436.080000,-0.050000,4000.000000,3853.200000,11.000000,28.000000,35.000000,103.000000,0.930000,1.000000,1.000000,1.000000,1.000000,0.200000,18.180000,43.000000,121.000000,1.0,0.0,0.0,1.000000,1.000000
75%,12.000000,663.000000,1.000000,662.910000,0.230000,4500.000000,4482.000000,14.000000,30.000000,40.000000,115.000000,1.160000,1.000000,1.000000,1.000000,1.000000,0.230000,43.060000,66.000000,202.000000,1.0,0.0,0.0,1.000000,1.000000
max,29.000000,3432.000000,6.000000,3431.840000,7.100000,6300.000000,6916.140000,23.000000,35.000000,67.000000,151.000000,2.410000,1.120000,1.080000,1.200000,1.200000,0.380000,413.870000,195.000000,302.000000,1.0,0.0,0.0,1.000000,1.440000


,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
0,Arbok,8.0,531.0,Hardy,1,Questions help guides,530.68,-0.32,Neutral,ArbokNeutral81,-,-,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.0,1.0,0.26,22.78,531.00,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
1,Arbok,9.0,574.0,Naughty,1,Rate My Mon,574.38,0.38,Naughty,ArbokNaughty91,Speed of Help,Main Skill Chance,3700.0,3276.72,Poison,Berries,Charge Energy S,14.0,32.0,40,Bean Sausage,103.0,1.09,0.90,1.00,1.0,0.8,0.26,22.78,538.38,ArbokNeutral91,80.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
2,Arbok,8.0,538.0,Quiet,1,Questions help guides,538.05,0.05,Quiet,ArbokQuiet81,Ingredient Finding,Exp Gains,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.2,1.0,0.26,22.78,530.63,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
3,Arcanine,5.0,958.0,Calm,3,,957.93,-0.07,Calm,ArcanineCalm53,Main Skill Chance,Speed of Help,2500.0,2728.00,Fire,Skills,Extra Helpful S,16.0,27.0,31,Fiery Herb,130.0,1.31,1.11,1.00,1.0,1.2,0.14,84.46,937.91,ArcanineNeutral53,31.0,130.0,1,True,,,,0.0,0.0,1.0,1.0
4,Arcanine,3.0,715.0,Lax,2,pokemon sleep general,714.96,-0.04,Lax,ArcanineLax32,Energy Recovery,Main Skill Chance,2500.0,2490.00,Fire,Skills,Extra Helpful S,16.0,27.0,29,Fiery Herb,130.0,1.44,1.00,1.08,1.0,0.8,0.14,61.16,750.12,ArcanineNeutral32,29.0,130.0,1,True,,,,0.0,0.0,1.0,1.0


In [5]:
# Load previous fit

x0, unpack_info = pack(make_initial_guess())
filename = f"./results/linear-fit-{digest(data, x0)}.npy"
opt = load(filename)
sol = unpack(opt.x, unpack_info)

opt

    message: `ftol` termination condition is satisfied.
    success: True
     status: 2
          x: [ 2.562e-01  2.545e-01 ...  2.196e-01  2.194e-01]
       cost: 814.695264281178
       grad: [-6.245e-07 -3.453e-05 ... -6.397e-05 -3.049e-05]
 optimality: 0.03631047531962395
       nfev: 58
       njev: 40

In [6]:
# Helpers

import scipy.stats as stats

def truncated_normal_sample(size, mu, sigma, lower, upper):
    return stats.truncnorm.rvs( (lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma, size=size)

def round_noise(size):
    eps = 1e-6
    return truncated_normal_sample(size, mu=0.0, sigma=0.25, lower= -0.5+eps, upper= 0.5-eps)

# This is an mean that gives more weigth to the smalled value
def harmonic(a,b):
    return 2.0 / ( 1.0 / a + 1.0 / b )

In [7]:
# In the boostrap method,
# We create N synthetic datasets that are likely to represent the population
# Then we redo the fit on those, starting from the best fit in initial dataset.
# Finally we collect statistics on the fitted variables between runs

# To create the synthetic datasets, we use sampling with replacement (allow duplicate)
# Because some pokemons are rare, we use stratified sampling

stratas = data.groupby(['Pokemon'], group_keys=False)
target_group_size =  stratas.size().quantile(q=0.75)
    

n_boostraps = 50
all_opt_x = [opt.x]

In [8]:
# WARNING: RUNING THE OPTIMISATION N TIMES IS VERY LONG.
#
# YOU CAN INTERRUPT THE KERNEL (STOP BUTTON) 
# AND RUN THE NEXT CELLS IF YOU WANT TO SEE CURRENT PROGRESS
#
# THE COLLECTION `all_opt_x` IS DEFINED ABOVE THIS CELL
# SO YOU CAN RESUME ADDING MORE RUNS

while( len(all_opt_x) < n_boostraps ):

    print( "Boostrap run " + str(len(all_opt_x)) )
    
    # To create the synthetic datasets, we use sampling with replacement (allow duplicate)
    
    # We'll introduce a correction torward more equal sample size between pokemons
    # We can motivate that by the fact stratified sampling should use population proportions 
    # instead of current data proportions
    
    resampled = stratas.apply(lambda x: x.sample( round( harmonic(len(x),target_group_size) ), replace=True, ignore_index=True))

    # Add small noise to RP
    # We do so to simulate some unknown value that would round() to current RP

    referenceRP = resampled["RP"].to_numpy()
    referenceRP += round_noise(len(resampled))
    
    # Compute per sample information about help time, nature, subskills etc
    
    recomputed = computedColumns(resampled)

    # Put the pieces together
    
    def residual(x):
        return referenceRP - computeRP(x, resampled, recomputed, unpack_info)
    
    # Redo the fit, starting from the optimal we found on current data
    opt2 = scipy.optimize.least_squares(residual, opt.x, loss="soft_l1", xtol=None, verbose=2) 
    
    # Collect results for stats
    all_opt_x.append(opt2.x)


Boostrap run 1
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         9.1264e+02                                    2.20e+06    


C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\least_squares.py:221: RuntimeWarning: overflow encountered in square
  z = (f / f_scale) ** 2


       1              8         9.0669e+02      5.95e+00       8.23e-02       9.66e+04    
       2              9         8.9132e+02      1.54e+01       2.06e-02       3.72e+05    
       3             11         8.8932e+02      2.00e+00       5.14e-03       3.98e+04    
       4             12         8.8884e+02      4.85e-01       1.03e-02       1.03e+03    
       5             13         8.8815e+02      6.85e-01       2.06e-02       1.20e+03    
       6             14         8.8714e+02      1.01e+00       4.12e-02       1.28e+03    
       7             15         8.8577e+02      1.37e+00       8.23e-02       1.82e+02    
       8             16         8.8438e+02      1.38e+00       1.65e-01       3.29e+02    
       9             17         8.8376e+02      6.24e-01       3.29e-01       6.69e+02    
      10             18         8.8372e+02      4.45e-02       6.59e-01       8.30e+00    
      11             20         8.8371e+02      4.67e-03       3.29e-01       1.13e+01    

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         9.3603e+02                                    2.73e+05    
       1              4         9.2904e+02      6.99e+00       1.66e-01       8.74e+04    
       2              6         9.1353e+02      1.55e+01       1.04e-02       4.50e+05    
       3              7         9.0836e+02      5.17e+00       2.07e-02       1.50e+05    
       4              8         9.0683e+02      1.53e+00       4.15e-02       1.73e+05    
       5              9         9.0560e+02      1.24e+00       4.15e-02       3.67e+04    
       6             10         9.0443e+02      1.16e+00       8.30e-02       1.14e+03    
       7             11         9.0321e+02      1.22e+00       1.66e-01       1.73e+03    
       8             12         9.0234e+02      8.71e-01       3.32e-01       2.59e+03    
       9             13         9.0150e+02      8.40e-01       6.64e-01       3.71e+03    

       1              9         9.3615e+02      1.54e+01       2.06e-02       7.86e+05    
       2             10         9.3154e+02      4.61e+00       2.06e-02       9.56e+05    
       3             12         9.2696e+02      4.58e+00       5.14e-03       9.21e+04    
       4             13         9.2532e+02      1.64e+00       1.03e-02       3.51e+04    
       5             14         9.2370e+02      1.62e+00       2.06e-02       4.44e+03    
       6             15         9.2188e+02      1.81e+00       4.12e-02       6.64e+02    
       7             16         9.2014e+02      1.75e+00       8.23e-02       2.85e+03    
       8             17         9.1849e+02      1.65e+00       1.65e-01       1.15e+02    
       9             18         9.1747e+02      1.02e+00       3.29e-01       8.85e+02    
      10             19         9.1663e+02      8.41e-01       6.59e-01       1.28e+03    
      11             20         9.1644e+02      1.82e-01       1.32e+00       6.03e+04    

...

       1              9         9.3300e+02      3.98e+01       2.06e-02       1.57e+06    
       2             10         9.2573e+02      7.27e+00       4.12e-02       3.23e+04    
       3             11         9.1668e+02      9.05e+00       8.23e-02       3.68e+04    
       4             12         9.1161e+02      5.07e+00       1.65e-01       1.97e+05    
       5             13         9.0832e+02      3.29e+00       3.29e-01       9.83e+04    
       6             14         9.0576e+02      2.56e+00       6.59e-01       2.13e+04    
       7             15         9.0376e+02      2.00e+00       1.32e+00       3.65e+03    
       8             16         9.0300e+02      7.57e-01       2.63e+00       2.37e+04    
       9             20         9.0296e+02      3.88e-02       8.23e-02       1.49e+02    
      10             22         9.0296e+02      3.93e-03       4.12e-02       2.28e+01    
      11             23         9.0295e+02      3.33e-03       8.23e-02       1.75e+01    

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_lsq\least_squares.py:221: RuntimeWarning: overflow encountered in square
  z = (f / f_scale) ** 2


       1              7         9.0244e+02      4.20e+00       3.29e-01       6.65e+05    
       2              9         8.8046e+02      2.20e+01       2.06e-02       1.63e+05    
       3             10         8.7531e+02      5.15e+00       4.12e-02       1.26e+05    
       4             11         8.7249e+02      2.82e+00       8.23e-02       1.63e+05    
       5             12         8.7184e+02      6.48e-01       8.23e-02       1.67e+05    
       6             13         8.7096e+02      8.76e-01       8.23e-02       1.18e+04    
       7             14         8.7038e+02      5.85e-01       1.65e-01       4.95e+03    
       8             15         8.6966e+02      7.21e-01       3.29e-01       9.40e+03    
       9             16         8.6896e+02      6.93e-01       6.59e-01       1.21e+04    
      10             17         8.6872e+02      2.45e-01       1.32e+00       8.47e+03    
      11             19         8.6867e+02      4.83e-02       6.59e-01       2.23e+02    

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.0030e+03                                    2.30e+06    
       1              4         9.7317e+02      2.98e+01       1.72e-01       2.20e+06    
       2              5         9.6629e+02      6.88e+00       3.45e-01       1.15e+05    
       3              6         9.6492e+02      1.37e+00       6.90e-01       1.11e+05    
       4              7         9.6454e+02      3.81e-01       1.38e+00       9.02e+03    
       5              8         9.6453e+02      6.52e-03       4.23e-01       2.56e+02    
       6              9         9.6453e+02      2.04e-06       6.23e-04       8.93e-01    
`ftol` termination condition is satisfied.
Function evaluations 9, initial cost 1.0030e+03, final cost 9.6453e+02, first-order optimality 8.93e-01.
Boostrap run 42
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0         

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         8.7280e+02                                    1.94e+06    
       1              6         8.4562e+02      2.72e+01       1.39e-02       4.35e+05    
       2              7         8.3840e+02      7.22e+00       2.79e-02       1.95e+05    
       3              8         8.3329e+02      5.11e+00       5.58e-02       4.72e+04    
       4              9         8.2956e+02      3.73e+00       1.12e-01       5.41e+03    
       5             10         8.2623e+02      3.33e+00       2.23e-01       2.00e+03    
       6             11         8.2382e+02      2.42e+00       4.46e-01       2.66e+04    
       7             12         8.2307e+02      7.46e-01       8.92e-01       1.62e+04    
       8             13         8.2306e+02      8.76e-03       5.11e-01       7.16e+01    
       9             14         8.2306e+02      7.66e-08       1.50e-03       2.63e-02    

In [11]:
# Stats
center = np.mean(all_opt_x, axis=0)
interval = 1.96 * np.std(all_opt_x, axis=0) # 95% confidence region

sol_center = unpack(center, unpack_info)
sol_interval = unpack(interval, unpack_info)

# Pretty display
pd.DataFrame({
    
    "Pokemon":game.pokedex.data["Pokemon"], 
    
    "ing%": sol_center["Pokemons ing fractions"]*100.0,
    "conf (ing%)": sol_interval["Pokemons ing fractions"]*100.0,
    
    "skill% * skillValue": sol_center["Pokemons skill products"], 
    "conf (skill% * skillValue)": sol_interval["Pokemons skill products"], 

}).set_index("Pokemon")

,ing%,conf (ing%),skill% * skillValue,conf (skill% * skillValue)
Pokemon,,,,
Bulbasaur,25.611921,0.065858,16.734934,0.085509
Ivysaur,25.452520,0.044513,16.697399,0.036660
Venusaur,26.573064,0.054338,18.419628,0.043677
Charmander,20.032967,0.047643,9.684981,0.059745
Charmeleon,22.662964,0.038804,14.046672,0.033176
Charizard,22.413154,0.045732,14.007450,0.042932
Squirtle,26.983311,0.061080,17.638532,0.078973
Wartortle,27.053646,0.040483,17.571616,0.033366
Blastoise,27.445990,0.062465,18.432321,0.046031
